In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler



from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import PowerTransformer


# Dataset Preparation
Read the dataset and clean datas. Remove outliers and transform features.
Create target with multi ouptu regressor problem.

In [2]:
# define constant & read the dataset
FOLDER = 'dataset/'
FILENAME = 'training_dataset.csv'

df = pd.read_csv(f'{FOLDER}{FILENAME}')
df.shape

(4371, 20)

In [3]:
df.columns.tolist()
df['matchdate']=pd.to_datetime(df['matchday'], format='%Y-%m-%d')
df

,Unnamed: 0,matchday,home_team,away_team,season,competition,correct_score,1N2_1,1N2_N,1N2_2,...,away_team_rank_FIFA,home_team_trend,away_team_trend,home_team_odd_trend,away_team_odd_trend,target,best_odd_won,corr_score_home,corr_score_away,matchdate
0,0,2002-05-31,France,Senegal,2002,world-cup,0:1,1.33,4.28,8.63,...,38.454545,-1.0,-1.0,-1.0000,-1.0000,2.0,False,0.999099,0.984973,2002-05-31
1,1,2002-06-01,Uruguay,Denmark,2002,world-cup,1:2,2.65,2.72,2.97,...,15.727273,-1.0,-1.0,-1.0000,4.3675,2.0,False,0.785714,0.988381,2002-06-01
2,2,2002-06-01,Ireland,Cameroon,2002,world-cup,1:1,3.33,2.90,2.28,...,19.090909,8.0,-1.0,2.1125,5.7825,1.0,False,0.881049,1.000000,2002-06-01
3,3,2002-06-01,Germany,Saudi Arabia,2002,world-cup,8:0,1.29,4.73,8.73,...,35.818182,10.0,10.0,2.5325,2.1125,0.0,True,0.989398,0.858464,2002-06-01
4,4,2002-06-02,Spain,Slovenia,2002,world-cup,3:1,1.42,4.10,7.57,...,30.909091,11.0,6.0,1.7250,2.2550,0.0,True,0.973360,0.952510,2002-06-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,4366,2021-07-03,Czech Republic,Denmark,2020,euro,1:2,3.94,3.08,2.13,...,10.666667,9.0,9.0,1.9425,2.0625,2.0,True,0.884615,0.988381,2021-07-03
4367,4367,2021-07-03,Ukraine,England,2020,euro,0:4,7.64,3.98,1.52,...,4.000000,12.0,12.0,1.8250,2.0300,2.0,True,0.956146,0.949018,2021-07-03
4368,4368,2021-07-06,Italy,Spain,2020,euro,2:1,2.50,3.14,3.10,...,6.000000,9.0,9.0,2.0100,1.9175,0.0,True,0.873981,0.973360,2021-07-06
4369,4369,2021-07-07,England,Denmark,2020,euro,2:1,1.75,3.44,5.58,...,10.666667,12.0,9.0,2.0675,2.1250,0.0,True,0.949018,0.988381,2021-07-07


In [4]:
features = ['1N2_1','1N2_N','1N2_2','away_team_rank_FIFA','home_team_rank_FIFA','season']
target = ['home_score','away_score']
for idx, r in df.iterrows():
    s = r.correct_score.split(':')
    df.loc[idx,'home_score']=int(s[0])
    df.loc[idx,'away_score']=int(s[1]) 
df.drop('correct_score', axis=1, inplace=True)    


In [5]:
trainset = df.query('competition != "world-cup" or (competition == "world-cup" and season < 2014)').drop('competition', axis=1)[features+target]
testset = df.query('competition == "world-cup" and (season <= 2014)').drop('competition', axis=1)[features+target]
unseenset = df.query('competition == "world-cup" and (season == 2018)').drop('competition', axis=1)[features+target]

wc2018 =df.query('competition == "world-cup" and (season == 2018)')



In [6]:
trainset.shape, testset.shape, unseenset.shape

((4291, 8), (193, 8), (64, 8))

In [7]:
trainset.dtypes, testset.dtypes, unseenset.dtypes

(1N2_1                  float64
 1N2_N                  float64
 1N2_2                  float64
 away_team_rank_FIFA    float64
 home_team_rank_FIFA    float64
 season                   int64
 home_score             float64
 away_score             float64
 dtype: object,
 1N2_1                  float64
 1N2_N                  float64
 1N2_2                  float64
 away_team_rank_FIFA    float64
 home_team_rank_FIFA    float64
 season                   int64
 home_score             float64
 away_score             float64
 dtype: object,
 1N2_1                  float64
 1N2_N                  float64
 1N2_2                  float64
 away_team_rank_FIFA    float64
 home_team_rank_FIFA    float64
 season                   int64
 home_score             float64
 away_score             float64
 dtype: object)

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
train_X = trainset.drop(['home_score','away_score'],axis=1).values
train_y = trainset[['home_score', 'away_score']].values


test_X = testset.drop(['home_score','away_score'],axis=1).values
test_y = testset[['home_score', 'away_score']].values


unseen_X = unseenset.drop(['home_score','away_score'],axis=1).values

wc2018.reset_index(inplace=True)
wc2018.index

WC_TABLE = 'wc_2018_table.json'
import pandas as pd
import json
import os


# Reading the json as a dict
with open('dataset/wc_2018_table.json') as json_data:
    data = json.load(json_data)
    
wc_table = pd.DataFrame.from_dict(data['stage'], orient='index').T.reset_index()
print(wc_table)

     index                                     groups
0  group-a     [Uruguay, Russia, Saudi Arabia, Egypt]
1  group-b           [Spain, Portugal, Iran, Morocco]
2  group-c        [France, Denemark, Peru, Australia]
3  group-d     [Croatia, Argentina, Nigeria, Iceland]
4  group-e  [Brazil, Switzerland, Costa Rica, Serbia]
5  group-f     [Sweden, Mexico, South Korea, Germany]
6  group-g        [Belgium, England, Panama, Tunisia]
7  group-h         [Columbia, Japan, Senegal, Poland]


In [10]:
def define_unknown_game(home, away):
    """_summary_
    Look inside the training_dataset for both teams games and fifa ranking and the collect
    same pattern game

    Args:
        home (_type_): _description_
        away (_type_): _description_
    """
    
    # get a list of ranking FIFA
    #print(home, away)
    rk = pd.read_csv('dataset/fifa_ranking-2021-05-27.csv')
    rk['rank_date']=pd.to_datetime(rk['rank_date'])
    rk['rank_year']=rk.rank_date.dt.year
    rk = rk.query('rank_year == 2018').copy()
    rk = rk.groupby('country_full').mean()

    ts = pd.read_csv('dataset/training_dataset.csv')
    ts['FIFA_diff']=abs(ts['home_team_rank_FIFA']-ts['away_team_rank_FIFA'])
    
    rk= rk.query('country_full == @home or country_full == @away')
    try:
        home_rank = rk.query('country_full == @home')['rank'][0]
        away_rank = rk.query('country_full == @away')['rank'][0]
    except:
        home_rank=0
        away_rank=0
        
    diff = abs(home_rank - away_rank)
    
    subset_ts = ts.query('(home_team == @home or home_team == @away) and (away_team == @home or away_team == @away)')
    if subset_ts.shape[0]==0:
        subset_ts=ts.query('(home_team_rank_FIFA > (@home_rank*0.75) and home_team_rank_FIFA < (@home_rank*1.25)) and  (away_team_rank_FIFA > (@away_rank*0.75) and away_team_rank_FIFA < (@away_rank*1.25)) ')
    if subset_ts.shape[0]==0:
        subset_ts = ts
    subset_ts= pd.DataFrame(subset_ts.mean()).T
    subset_ts['matchday']=None
    subset_ts['home_team']=home
    subset_ts['away_team']=away
    subset_ts['correct_score']='0:0'
    try:
        return rk.query('country_full == @home')['rank'][0],rk.query('country_full == @away')['rank'][0], subset_ts
    except:
        #print('exception ............')
        #print(subset_ts.fillna(0,))
        return (0),(0),subset_ts.fillna(0)


In [11]:
winners2=[]

import warnings
warnings.filterwarnings("ignore")
from sklearn.exceptions import FitFailedWarning 

import joblib


regressor = DecisionTreeRegressor()



steps = [('scaler', StandardScaler()),('power',PowerTransformer()) ,('REG', regressor)]

pipeline = Pipeline(steps)

parameteres = {'REG__max_features':['log2'],'REG__max_depth':[1,2,3,5],'REG__min_weight_fraction_leaf':[0.1,0.4,0.7,1]}

parameteres = {'REG__max_features':['log2']}



grid = GridSearchCV(pipeline, param_grid=parameteres, cv=10)



for j in range(0,1500):
    
    grid.fit(np.abs(train_X), train_y)
    #print(grid.score(np.abs(test_X),test_y))
    #print(grid.best_params_)


    #save your model or results
    joblib.dump(grid,'models/multi-regressor-score.pkl')

    #load your model for further usage
    model = joblib.load("models/multi-regressor-score.pkl")



    group_stage=model.predict(unseen_X).astype(int)
    for i in range(0,48):
        #
        # print(group_stage[i][0], group_stage[i][1])
        wc2018.loc[i,'pred_home_score']= group_stage[i][0]
        wc2018.loc[i,'pred_away_score']= group_stage[i][1]
        if group_stage[i][0] > group_stage[i][1]:
            wc2018.loc[i,'h_points']=3
            wc2018.loc[i,'a_points']=0
        elif group_stage[i][0] == group_stage[i][1]:
            wc2018.loc[i,'h_points']=1
            wc2018.loc[i,'a_points']=1
        else:
            wc2018.loc[i,'h_points']=0
            wc2018.loc[i,'a_points']=3
        
    disp_cols =['home_team','away_team','home_score','away_score','pred_home_score','pred_away_score','h_points','a_points']        
    table = pd.DataFrame()
    for idx, r in wc_table.iterrows():
        teams = r.groups
        group = r['index']
        for team in teams:
            # look in predict table for point and score
            h=wc2018.query('home_team == @team')[['home_score','away_score','h_points']]
            h.columns=['for','against','points']
            a=wc2018.query('away_team == @team')[['away_score','home_score','a_points']]
            a.columns=['for','against','points']

            frames = [a,h]
            t=pd.DataFrame(pd.concat(frames).sum()).T
            t['team']=team      
            t['group']=group
            
            frames = [table, t]
            table = pd.concat(frames)  
            


    cols = ['group','team','for','against','diff','points']
    table['diff']= table['for']-table['against']
    table=table[cols].sort_values(['group','points','diff','for','against'], ascending=[True,False,False,False,True])
    table
    first_in_group = table.groupby(['group']).nth(0).copy()
    for idx, r in first_in_group.iterrows():
        grp = idx.split('-')
        code = f"1{grp[1].upper()}"
        first_in_group.loc[idx,'code']=code


    second_in_group = table.groupby(['group']).nth(1).copy()
    for idx, r in second_in_group.iterrows():
        grp = idx.split('-')
        code = f"2{grp[1].upper()}"
        second_in_group.loc[idx,'code']=code
            
    frames = [first_in_group,second_in_group]
    next_game = pd.concat(frames)
    next_game

    ko16 = pd.DataFrame.from_dict(data['knockouts'], orient='index').T.reset_index()
    ko16
    
    games16 = []
    for idx, r in ko16.iterrows():
        g1 = r['knockout-16'][0]
        g2 = r['knockout-16'][1]
        
        t1 = g1[0]
        t2 = g1[1]
        
        games16.append([next_game.query('code == @t1').team.values[0],next_game.query('code == @t2').team.values[0]])
        
        t1 = g2[0]
        t2 = g2[1]
        
        games16.append([next_game.query('code == @t1').team.values[0],next_game.query('code == @t2').team.values[0]])
        
    games16
    
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    ko16_game = []
    pko = []
    for game in games16:
        t1 = game[0]
        t2 = game[1]
        #g = alg.query('(home_team == @t1) and (away_team == @t2)').index
        #print(g, len(g))
        # if len(g) >1 then call PredictGame with data
        rh, ra, ts = define_unknown_game(t1,t2)
        #print(ts.shape)
        
        p=model.predict(ts[features].values.reshape(1,-1)).astype(int)
        if p[0][0] == p[0][1]:
            result = 'draw'
            if rh < ra:
                team=t1
            else:
                team=t2
        else:
            result='winner'
            if p[0][0]>p[0][1]:
                team = t1
            else:
                team=t2
        try:
            ko16_game.append([t1,t2,p,result, rh.astype(int), ra.astype(int), team])
        except:
            ko16_game.append([t1,t2,p,result, rh, ra, team])
            
        
    ko16_game
    
    
    temp=pd.DataFrame(ko16_game, columns=['home','away','pred_score','res','h_rank','a_rank', 'winner'])
    q_final = temp.winner.tolist()
    hq=q_final[::2]
    for h in hq:
        q_final.remove(h)

    q_final_game = []    
    for x in range(0,4):
        q_final_game.append([q_final[x], hq[x]])

    games08 = q_final_game    
    q_final_game

    ko08_game = []
    pko = []
    for game in games08:
        t1 = game[0]
        t2 = game[1]
        #g = alg.query('(home_team == @t1) and (away_team == @t2)').index
        #print(g, len(g))
        # if len(g) >1 then call PredictGame with data
        rh, ra, ts = define_unknown_game(t1,t2)
        #print(ts[features])
        
        p=model.predict(ts[features].values.reshape(1,-1)).astype(int)
        if p[0][0] == p[0][1]:
            result = 'draw'
            if rh < ra:
                team=t1
            else:
                team=t2
        else:
            result='winner'
            if p[0][0]>p[0][1]:
                team = t1
            else:
                team=t2
        try:
            ko08_game.append([t1,t2,p,result, rh.astype(int), ra.astype(int), team])
        except:
            ko08_game.append([t1,t2,p,result, rh, ra, team])
            
        
    ko08_game
        
    temp=pd.DataFrame(ko08_game, columns=['home','away','pred_score','res','h_rank','a_rank', 'winner'])
    q_final = temp.winner.tolist()
    hq=q_final[::2]
    for h in hq:
        q_final.remove(h)

    q_final_game = []    
    for x in range(0,2):
        q_final_game.append([q_final[x], hq[x]])

    games04 = q_final_game    
    q_final_game
    
    ko04_game = []
    pko = []
    for game in games04:
        t1 = game[0]
        t2 = game[1]
        #g = alg.query('(home_team == @t1) and (away_team == @t2)').index
        #print(g, len(g))
        # if len(g) >1 then call PredictGame with data
        rh, ra, ts = define_unknown_game(t1,t2)
        #print(ts[features])
        
        p=model.predict(ts[features].values.reshape(1,-1)).astype(int)
        if p[0][0] == p[0][1]:
            result = 'draw'
            if rh < ra:
                team=t1
            else:
                team=t2
        else:
            result='winner'
            if p[0][0]>p[0][1]:
                team = t1
            else:
                team=t2
        try:
            ko04_game.append([t1,t2,p,result, rh.astype(int), ra.astype(int), team])
        except:
            ko04_game.append([t1,t2,p,result, rh, ra, team])
            
        
    ko04_game
    
    
    t1=ko04_game[0][6]
    t2=ko04_game[1][6]
    rh, ra, ts = define_unknown_game(t1,t2)
    p=model.predict(ts[features].values.reshape(1,-1)).astype(int)
    if p[0][0] == p[0][1]:
        result = 'draw'
        if rh < ra:
            team=t1
            looser=t2
        else:
            team=t2
            looser=t1
    else:
        result='winner'
        if p[0][0]>p[0][1]:
            team = t1
            losser=t2
        else:
            team=t2
            looser=t1
            
    #print(f'WINNER is {team}')
    winners2.append([team,p[0][0],p[0][1], looser, grid.score(test_X, test_y)])


In [17]:
grid.best_params_

{'REG__max_features': 'log2'}

In [16]:
winners=pd.DataFrame(winners2, columns=['winner','score_h','score_a','looser','score'])
winners.winner.value_counts(normalize=True)

Brazil          0.142000
Belgium         0.113333
France          0.110667
Spain           0.108667
England         0.095333
Portugal        0.088000
Switzerland     0.058667
Argentina       0.039333
Peru            0.034000
Uruguay         0.033333
Germany         0.032667
Croatia         0.026000
Sweden          0.023333
Russia          0.015333
Senegal         0.012667
Nigeria         0.012000
Tunisia         0.009333
Mexico          0.007333
Serbia          0.007333
South Korea     0.005333
Morocco         0.004667
Iceland         0.004000
Poland          0.004000
Australia       0.003333
Japan           0.003333
Iran            0.002667
Costa Rica      0.001333
Egypt           0.001333
Saudi Arabia    0.000667
Name: winner, dtype: float64

In [18]:
pd.DataFrame(winners[['winner','score']].groupby('winner').sum()).sort_values('score',ascending=False)

,score
winner,
Brazil,169.201627
Belgium,134.989408
France,132.381613
Spain,129.724451
England,113.133785
Portugal,104.828592
Switzerland,70.225606
Argentina,46.885108
Peru,40.099342


In [19]:
winners.shape


(1500, 5)

In [21]:
winners.to_csv('data/winners1500.csv')